In [ ]:
!git clone https://github.com/derInformatiker/AIcrowd-AIBlitz7-Solution.git
!pip install -r AIcrowd-AIBlitz7-Solution/challenge5/requirements.txt
!pip install aicrowd-cli==0.1
!pip install trianglesolver==1.2

###RESTART RUNTIME TO USE NEW PACKAGES

In [2]:
API_KEY = ""  # Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [ ]:
!aicrowd dataset download --challenge image-correction
!rm -rf data
!mkdir data

!unzip -q test.zip  -d data/test
!unzip -q train.zip  -d data/train

In [10]:
import shutil

shutil.copy('AIcrowd-AIBlitz7-Solution/challenge5/load.py','load.py')
shutil.copy('AIcrowd-AIBlitz7-Solution/challenge5/matching.py','matching.py')

'matching.py'

In [11]:
import cv2, os
import math
from glob import glob
import numpy as np
from trianglesolver import solve, degree
import random
from load import *
from matching import *

In [12]:
def run2(mode, d,out = np.array([])):
    (start, last, label), image, black = preprocess(mode,d)
    y, x = start.shape[:2]
    x_num = int(512/x)
    y_num = int(512/y)
    if out.shape[0] == 0:
        out = np.zeros((x_num,y_num,y,x,3))

        start_right, image, _ = findN2(start,image,2,right)
        start_bottom, image, _ = findN2(start,image,2,bottom)

        last_top, image, _ = findN2(last,image,2,top)
        last_left, image, _ = findN2(last,image,2,left)

        out[0][0] = start
        out[-1][-1] = last
        out[0][1] = start_bottom
        out[1][0] = start_right
        out[-1][-2] = last_top
        out[-2][-1] = last_left

    if image.shape[0] != 0:
        for _ in range(4):
            filled, nCounts = getNei(out)
            nei = np.zeros(nCounts.shape)
            nei[nCounts > 1] = 1

            for i in range(x_num):
                for j in range(y_num):
                    if nei[i][j] == 1:
                        p_images = []
                        scores = []
                        remove_index = []
                        for d in range(4):
                            try:
                                img = getImgNei(out,i,j,d)
                                out_image, image, confidence, removeIndex = findN2(img,image,2,(d+2)%4,remove = False)
                                if d == top or d == bottom:
                                    confidence = confidence
                                p_images.append(out_image)
                                scores.append(confidence)
                                remove_index.append(removeIndex)
                            except (IndexError,ValueError):
                                pass
                        index = scores.index(min(scores))
                        p_image =  p_images[index]
                        out[i][j] =  p_images[index]
                        image = np.delete(image,remove_index[index],axis = 0)
    img = arangeNew(out)
    return img, label

def run3(mode, d,out = np.array([])):
    (start, last, label), image, black = preprocess(mode,d)
    y, x = start.shape[:2]
    x_num = int(512/x)
    y_num = int(512/y)
    if out.shape[0] == 0:
        out = np.zeros((x_num,y_num,y,x,3))

        start_right, image, _ = findN2(start,image,2,right)
        start_bottom, image, _ = findN2(start,image,2,bottom)

        last_top, image, _ = findN2(last,image,2,top)
        last_left, image, _ = findN2(last,image,2,left)

        out[0][0] = start
        out[-1][-1] = last
        out[0][1] = start_bottom
        out[1][0] = start_right
        out[-1][-2] = last_top
        out[-2][-1] = last_left

    if image.shape[0] != 0:
        for _ in range(4):
            filled, nCounts = getNei(out)
            nei = np.zeros(nCounts.shape)
            nei[nCounts > 0] = 1

            for i in range(x_num):
                for j in range(y_num):
                    if nei[i][j] == 1:
                        p_images = []
                        scores = []
                        remove_index = []
                        for d in range(4):
                            try:
                                img = getImgNei(out,i,j,d)
                                out_image, image, confidence, removeIndex = findN2(img,image,2,(d+2)%4,remove = False)
                                if d == top or d == bottom:
                                    confidence = confidence
                                p_images.append(out_image)
                                scores.append(confidence)
                                remove_index.append(removeIndex)
                            except (IndexError,ValueError):
                                pass
                        index = scores.index(min(scores))
                        p_image =  p_images[index]
                        out[i][j] =  p_images[index]
                        image = np.delete(image,remove_index[index],axis = 0)
    img = arangeNew(out)
    return img, label

In [13]:
def run(mode, d,out = np.array([])):
    (start, last, label), imagee, black = preprocess(mode,d)
    y, x = start.shape[:2]
    x_num = int(512/x)
    y_num = int(512/y)
    
    image = imagee.copy()
    start_down, image = findN(start,image,x_num,right,asList = True)
    outs = []
    for starting_image in start_down:
        out, image = findN(starting_image,image,y_num,bottom,last)
        outs.append(out)
    h = np.concatenate((outs),axis = 1)
    
    image = imagee.copy()
    start_down, image = findN(start,image,y_num,bottom,asList = True)
    outs = []
    for starting_image in start_down:
        out, image = findN(starting_image,image,x_num,right,last)
        outs.append(out)
    v = np.concatenate((outs),axis = 0) 
    
    s = run2(mode,d)[0]
    f = run3(mode,d)[0]
    out = np.mean((h,v,s,f),axis = 0).astype('uint8')
    #out = s
    img = fillup(out)
    
    return img, label

In [16]:
if not os.path.exists('submission'):
    os.makedirs('submission/Labels')

for i in range(5000):
    out, label = run('test',i)
    (start, last, label), image, black = preprocess('train',i)
    out = np.mean((out,label),axis = 0)
    out = cv2.cvtColor(out.astype(np.uint16),cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'submission/Labels/{i}.jpg',out,[int(cv2.IMWRITE_JPEG_QUALITY), 50])